Welcome! This IPython notebook does a sensitivity analysis of the mobility model for the water access research project.

Import Packages

In [65]:
import pandas as pd
import mobility_module as mm
import water_access_tools as wtools
import numpy as np
import plotly
import plotly.express as px
import plotting_tools_water_access
from plotly.subplots import make_subplots
import plotly.graph_objects as go



Import data from csv files

In [66]:
######################
#### Import Data #####
# with open("../data/mobility-model-parameters.csv") as csv_file:
#     # read the csv file
#     allHPV_param_df = pd.read_csv("../data/mobility-model-parameters.csv")

allHPV_param_df = wtools.load_data("mobility-model-parameters.csv")

sens_df = wtools.load_data("Sensitivity Analysis Variables.csv")

country_data = wtools.load_data("country_data_master_interpolated.csv")

assert country_data["alpha3"].value_counts().max() == 1, "Duplicate alpha3 codes"

# set country_data index to alpha3
country_data = country_data.set_index("alpha3")



Initiliase Velocity Kgs, define colours for graphing

In [67]:
# add power values to country_data
insufficient_exercise_series = 100 - country_data["percent_insufficient_activity"]
country_data["power"] = wtools.map_values_std_dev(insufficient_exercise_series, power_max = 90.75, power_min = 59.25, scale_factor = 3)


# map road values to Crr per country
road_disrepair = 7 - country_data["RoadQuality"]
country_data["Crr"] = wtools.map_values_std_dev(road_disrepair, power_max = 0.03, power_min = 0.002, scale_factor = 3)



In [68]:
graph_colours = ["#3D87CB", "#F0B323", "#DC582A", "#674230", "#3A913F", "#75787B"]
full_result_dict = {}
plot_dict = {}
df_large = pd.DataFrame()


### FIlter Data
col = "Name"
filter_value = "Bicycle"
param_df = allHPV_param_df.loc[(allHPV_param_df[col] == filter_value)]

for country_code in country_data.index:


    ###### CHNAGE THESE SOON
    country_Crr = country_data.loc[country_code, "Crr"]
    country_human_weight = country_data.loc[country_code, "Average Weight"]
    country_power_output = country_data.loc[country_code, "power"]

    # # initialise variables, options, MET, hpv and Velocity Kgs classes, populated with data via 'init' functions
    mo = mm.model_options()
    mv = mm.model_variables()
    met = mm.MET_values(mv)
    hpv = mm.HPV_variables(param_df, mv)
    mr = mm.model_results(hpv, mo)

    # select model (bike...)
    mo.model_selection = 2

    trip_velocity_mean = []  # create empty list to place variables in to in loop
    unloaded_velocity_mean = []  # create empty list to place variables in to in loop
    loaded_velocity_mean = []  # create empty list to place variables in to in loop
    velocitykgs = []  # create empty list to place variables in to in loop
    water_ration_kms = []  # create empty list to place variables in to in loop


    # "Coefficient of Rolling Resistance":
    hpv.Crr = np.array([[[country_Crr]]]) # change
    # "Human Weight":
    mv.m1 = country_human_weight 
    # "Human Power Output":
    mv.P_t = country_power_output

    ######## Numerical MODEL ########
    mr.v_load_matrix3d, mr.load_matrix3d = mm.mobility_models.numerical_mobility_model(
        mr, mv, mo, met, hpv
    )
    ####### Organise Velocity Kgs #######
    mr.load_results(hpv, mv, mo)

    maximum_vel_kg_per_slope = np.amax(mr.velocitykgs, axis=2)
    mean_vel_kg_per_slope = np.mean(maximum_vel_kg_per_slope)

    # append to list
    trip_velocity_mean.append(np.mean(mr.v_avg_matrix3d))
    unloaded_velocity_mean.append(np.mean(mr.v_unload_matrix3d))
    loaded_velocity_mean.append(np.mean(mr.v_load_matrix3d))
    velocitykgs.append(mean_vel_kg_per_slope)
    water_ration_kms.append(mean_vel_kg_per_slope/mv.waterration*mr.t_secs/1000) # [kg][m/s] * [s] * 1/[kg] * 1/1000 = [km]. Divide by distance to water, and accomodate for household size to see if needs are met.

    # create dictionary of results
    result_dict = {
        "trip_velocity_mean": trip_velocity_mean[0],
        "unloaded_velocity_mean": unloaded_velocity_mean[0],
        "loaded_velocity_mean": loaded_velocity_mean[0],
        "velocitykgs": velocitykgs[0],
        "water_ration_kms": water_ration_kms[0],
    }
    
    # append to dictionary, with country code as key
    full_result_dict[country_code] = result_dict




In [71]:
# create dataframe from dictionary, with country code as index
full_result_dict_df = pd.DataFrame.from_dict(full_result_dict, orient="index")

# merge with country_data
country_data = country_data.merge(full_result_dict_df, left_index=True, right_index=True)

# export to csv
country_data.to_csv("../data/country_data_with_cycling_distance.csv")